In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp "/content/gdrive/My Drive/DataAugumentation.zip" .
!unzip -qq DataAugumentation.zip
!rm DataAugumentation.zip
data_path = 'DataAugumentation'

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import mean_squared_error
import joblib

import tensorflow as tf
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention, Embedding, GlobalAveragePooling1D

SEQUENCE_LENGTH = 600
BATCH_SIZE = 64
EPOCHS = 50
LEARNING_RATE = 1e-4
PLOT_SAVE_DIR = 'predicted_vs_actual_plots'

def process_file(file_path):
    df = pd.read_csv(file_path)
    df['Time'] = df['Time'] - df['Time'].iloc[0]
    df['Momentary fuel consumption'] = df['Trip fuel consumption'].diff().fillna(0)
    features = df[['Engine speed', 'Throttle position', 'Accelerator pedal position', 'Speed']]
    target = df['Momentary fuel consumption']
    features = features.iloc[:SEQUENCE_LENGTH]
    target = target.iloc[:SEQUENCE_LENGTH]
    return features.values, target.values

def pad_and_normalize(data, scaler, sequence_length=SEQUENCE_LENGTH):
    padded_data = pad_sequences(data, maxlen=sequence_length, dtype='float32', padding='post', truncating='post')
    normalized_data = scaler.transform(padded_data.reshape(-1, padded_data.shape[-1])).reshape(padded_data.shape)
    return normalized_data

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_original = []
y_original = []
X_augmented = []
y_augmented = []



Mounted at /content/gdrive


FileNotFoundError: [Errno 2] No such file or directory: '/content/DataAugumentation/original'

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import mean_squared_error
import joblib

SEQUENCE_LENGTH = 600
BATCH_SIZE = 64
EPOCHS = 50
LEARNING_RATE = 1e-4
PLOT_SAVE_DIR = 'predicted_vs_actual_plots'


def process_file(file_path):
    df = pd.read_csv(file_path)

    df['Time'] = df['Time'] - df['Time'].iloc[0]

    df['Momentary fuel consumption'] = df['Trip fuel consumption'].diff().fillna(0)

    features = df[['Engine speed', 'Throttle position', 'Accelerator pedal position', 'Speed']]
    target = df['Momentary fuel consumption']

    features = features.iloc[:SEQUENCE_LENGTH]
    target = target.iloc[:SEQUENCE_LENGTH]

    return features.values, target.values



def pad_and_normalize(data, scaler, sequence_length=SEQUENCE_LENGTH):
    padded_data = pad_sequences(data, maxlen=sequence_length, dtype='float32', padding='post', truncating='post')
    normalized_data = scaler.transform(padded_data.reshape(-1, padded_data.shape[-1])).reshape(padded_data.shape)
    return normalized_data


scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_original = []
y_original = []
X_augmented = []
y_augmented = []

base_folder_path = '/content/'
for i in range(5):
    folder_path = os.path.join(base_folder_path, f'data_aug(3_slices_with_repeated)_cluster_{i}')
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            features, target = process_file(file_path)

            slices = filename.split('_')
            is_original_trip = slices[0] == slices[2] and slices[0] == slices[4]

            if is_original_trip:
                X_original.append(features)
                y_original.append(target)
            else:
                X_augmented.append(features)
                y_augmented.append(target)

# Pad and convert lists to numpy arrays
X_original = pad_sequences(X_original, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')
y_original = pad_sequences(y_original, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')
X_augmented = pad_sequences(X_augmented, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')
y_augmented = pad_sequences(y_augmented, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')

num_test = int(0.2 * len(X_original))
X_test = X_original[:num_test]
y_test = y_original[:num_test]
X_train = np.concatenate([X_original[num_test:], X_augmented])
y_train = np.concatenate([y_original[num_test:], y_augmented])

scaler_X.fit(X_train.reshape(-1, X_train.shape[-1]))
scaler_y.fit(y_train.reshape(-1, 1))

joblib.dump(scaler_X, 'scaler_X.pkl')
joblib.dump(scaler_y, 'scaler_y.pkl')

['scaler_y.pkl']

In [ ]:
X_train_normalized = pad_and_normalize(X_train, scaler_X)
y_train_normalized = scaler_y.transform(y_train.reshape(-1, 1)).reshape(y_train.shape)
X_test_normalized = pad_and_normalize(X_test, scaler_X)
y_test_normalized = scaler_y.transform(y_test.reshape(-1, 1)).reshape(y_test.shape)

print('X_train_normalized shape:', X_train_normalized.shape)
print('y_train_normalized shape:', y_train_normalized.shape)
print('X_test_normalized shape:', X_test_normalized.shape)
print('y_test_normalized shape:', y_test_normalized.shape)

X_train_normalized shape: (28713, 600, 4)
y_train_normalized shape: (28713, 600)
X_test_normalized shape: (15, 600, 4)
y_test_normalized shape: (15, 600)


In [16]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer, Dense, Dropout, MultiHeadAttention, LayerNormalization
from tensorflow.keras.optimizers import Adam

# Constants
SEQUENCE_LENGTH = 600
FEATURE_DIM = 4
NUM_HEADS = 8
FF_DIM = 32
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 0.001

# Custom Positional Encoding Layer
class PositionalEncoding(Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.sequence_length = sequence_length
        self.output_dim = output_dim
        self.pos_encoding = self.positional_encoding(sequence_length, output_dim)

    def get_angles(self, pos, i, d_model):
        angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
        return pos * angle_rates

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(np.arange(position)[:, np.newaxis],
                                     np.arange(d_model)[np.newaxis, :],
                                     d_model)
        sines = np.sin(angle_rads[:, 0::2])
        cosines = np.cos(angle_rads[:, 1::2])
        pos_encoding = np.concatenate([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[np.newaxis, ...]
        return tf.cast(pos_encoding, dtype=tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

# Custom Transformer Block Layer
class TransformerBlock(Layer):
    def __init__(self, num_heads, ff_dim, rate=0.2):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=ff_dim)
        self.ffn = Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(FEATURE_DIM)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Building the model
model = Sequential([
    tf.keras.layers.InputLayer(shape=(SEQUENCE_LENGTH, FEATURE_DIM)),
    PositionalEncoding(SEQUENCE_LENGTH, FEATURE_DIM),
    TransformerBlock(NUM_HEADS, FF_DIM),
    TransformerBlock(NUM_HEADS, FF_DIM),
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='mean_squared_error')

# Show the model summary
model.summary()


model.fit(X_train_normalized, y_train_normalized, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.2)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ positional_encoding_1                │ (None, 600, 4)              │               0 │
│ (PositionalEncoding)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block (TransformerBlock) │ (None, 600, 4)              │           5,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block_1                  │ (None, 600, 4)              │           5,176 │
│ (TransformerBlock)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 600, 1)              │               5 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,357 (40.46 KB)

 Trainable params: 10,357 (40.46 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 84s 98ms/step - loss: 0.1333 - val_loss: 0.0104
Epoch 2/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 58s 81ms/step - loss: 0.0112 - val_loss: 0.0104
Epoch 3/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 82s 81ms/step - loss: 0.0110 - val_loss: 0.0103
Epoch 4/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 82s 81ms/step - loss: 0.0104 - val_loss: 0.0077
Epoch 5/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 56s 78ms/step - loss: 0.0086 - val_loss: 0.0069
Epoch 6/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 85s 82ms/step - loss: 0.0080 - val_loss: 0.0069
Epoch 7/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 82s 81ms/step - loss: 0.0078 - val_loss: 0.0068
Epoch 8/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 82s 81ms/step - loss: 0.0077 - val_loss: 0.0069
Epoch 9/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 82s 81ms/step - loss: 0.0076 - val_loss: 0.0069
Epoch 10/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 56s 78ms/step - loss: 0.0075 - val_loss: 0.0068


In [17]:
model.save('modelTransformer.h5')

In [18]:
test_loss = model.evaluate(X_test_normalized, y_test_normalized)
print(f"Test Loss: {test_loss}")

y_pred_normalized = model.predict(X_test_normalized)

y_test_inv = scaler_y.inverse_transform(y_test_normalized.reshape(-1, SEQUENCE_LENGTH))
y_pred_inv = scaler_y.inverse_transform(y_pred_normalized.reshape(-1, SEQUENCE_LENGTH))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0026
Test Loss: 0.002647922607138753
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [19]:
if not os.path.exists(PLOT_SAVE_DIR):
    os.makedirs(PLOT_SAVE_DIR)

for i in range(len(X_test_normalized)):
    # print(y_test_inv[i])
    # print(y_pred_inv[i])
    # input()
    cumulative_actuals = np.cumsum(y_test_inv[i], axis=0)
    cumulative_predictions = np.cumsum(y_pred_inv[i], axis=0)
    plt.figure(figsize=(12, 6))
    plt.plot(cumulative_actuals, label='Actual Trip Fuel Consumption')
    plt.plot(cumulative_predictions, label='Predicted Trip Fuel Consumption')
    plt.title(f'Trip {i + 1}: Actual vs Predicted Fuel Consumption')
    plt.xlabel('Time Steps')
    plt.ylabel('Trip Fuel Consumption (uL)')
    plt.legend()
    plot_filename = os.path.join(PLOT_SAVE_DIR, f'trip_{i + 1}_actual_vs_predicted.png')
    plt.savefig(plot_filename)
    plt.close()

!zip -r data.zip predicted_vs_actual_plots

updating: predicted_vs_actual_plots/ (stored 0%)
updating: predicted_vs_actual_plots/trip_7_actual_vs_predicted.png (deflated 9%)
updating: predicted_vs_actual_plots/trip_3_actual_vs_predicted.png (deflated 9%)
updating: predicted_vs_actual_plots/trip_6_actual_vs_predicted.png (deflated 9%)
updating: predicted_vs_actual_plots/trip_14_actual_vs_predicted.png (deflated 9%)
updating: predicted_vs_actual_plots/trip_10_actual_vs_predicted.png (deflated 11%)
updating: predicted_vs_actual_plots/trip_15_actual_vs_predicted.png (deflated 9%)
updating: predicted_vs_actual_plots/trip_5_actual_vs_predicted.png (deflated 9%)
updating: predicted_vs_actual_plots/trip_12_actual_vs_predicted.png (deflated 9%)
updating: predicted_vs_actual_plots/trip_1_actual_vs_predicted.png (deflated 10%)
updating: predicted_vs_actual_plots/trip_11_actual_vs_predicted.png (deflated 9%)
updating: predicted_vs_actual_plots/trip_4_actual_vs_predicted.png (deflated 9%)
updating: predicted_vs_actual_plots/trip_8_actual_vs_